# Overview

In [1]:
import sys

assert sys.version_info > (3, 7)

sys.version_info

In [2]:
from dataclasses import dataclass

@dataclass
class InventoryItem:
    """Class for keeping track of an item in inventory."""
    name: str
    unit_price: float
    quantity_on_hand: int = 0

    def total_cost(self) -> float:
        return self.unit_price * self.quantity_on_hand

In [3]:
try:
    item = InventoryItem()
except TypeError as e:
    print(repr(e))

In [4]:
item = InventoryItem("Widget", 1.99)

item

InventoryItem(name='Widget', unit_price=1.99, quantity_on_hand=0)

In [5]:
item.name

'Widget'

In [6]:
item.unit_price = 2.99

item

InventoryItem(name='Widget', unit_price=2.99, quantity_on_hand=0)

# Type Annotations

In [7]:
assert sys.version_info > (3, 6)

In [8]:
def greet(name: str):
    print("Hello, " + name)
    
greet("Jace")

Hello, Jace


In [9]:
## Argument Annotations

In [10]:
greet

<function __main__.greet(name: str)>

In [11]:
greet.__annotations__

{'name': str}

In [12]:
import typing  # stdlib

hints = typing.get_type_hints(greet)

hints  # `dict` with real classes

{'name': str}

In [13]:
import inspect  # stdlib

signature = inspect.signature(greet)  

signature  # `Signature` object

<Signature (name: str)>

In [14]:
signature.parameters

mappingproxy({'name': <Parameter "name: str">})

In [15]:
signature.parameters['name']

<Parameter "name: str">

In [16]:
# KEYWORD_ONLY: greet(name)
# POSITIONAL_ONLY: greet(*, name)
# VAR_POSITIONAL: greet(*names)
# VAR_KEYWORD: greet(**names)

signature.parameters['name'].kind

<_ParameterKind.POSITIONAL_OR_KEYWORD: 1>

In [17]:
signature.parameters['name'].annotation

str

### Return Annotations

In [18]:
from decimal import Decimal

def add_tax(subtotal, rate=0.06) -> Decimal:
    cents = Decimal('0.01')
    return Decimal(subtotal * (1 + rate)).quantize(cents)

add_tax(4.99)

Decimal('5.29')

In [19]:
inspect.signature(add_tax).return_annotation

decimal.Decimal

### Variable Annotations

In [20]:
class Person:
    name: str

In [21]:
Person.__annotations__

{'name': str}

### Optional Values

In [22]:
from typing import Optional

def fill(password: Optional[str]):
    if password is not None:
        ...
        
fill("abc123")
fill(None)

### Homogeneous Lists 

In [23]:
from typing import List

def print_one_more_than(numbers: List[int]):
    for number in numbers:
        print(number + 1)

### Mixed Types

In [24]:
from typing import Union

def print_items_or_keys(values: Union[list, dict]):
    for value in values:
        print(value)

### ⚠️ Circular Annotations

In [25]:
class Node:
    
    def connect_edge(edge: 'Edge'):
        pass
    

class Edge:
    def connect_node(node: Node):
        pass

In [26]:
from __future__ import annotations  


class Node:
    
    def connect_edge(edge: Edge):
        pass
     

class Edge:
    def connect_node(node: Node):
        pass

# Type Checking (with mypy)

In [27]:
# pip install mypy==0.670

In [28]:
from mypy import api

def mypy(*filenames):
    """Emulate `$ mypy <filename>` for notebooks."""
    message, _, _ = api.run(filenames)
    print(message or "(no errors)")

In [29]:
%%writefile greet.py

def greet(name: str):
    print("Hello, " + name)
    
greet("Jace")

Overwriting greet.py


In [30]:
mypy('greet.py')

(no errors)


In [31]:
%%writefile greet2.py

def greet(name: str):
    print("Hello, " + name)
    
greet(42)

Overwriting greet2.py


In [32]:
mypy('greet2.py')

greet2.py:5: error: Argument 1 to "greet" has incompatible type "int"; expected "str"



In [33]:
%%writefile people.py

from typing import Iterable, List


class Person:
    
    def __init__(self, name):
        self.name = name

        
def get_people(*names: Iterable[str]) -> List[Person] :
    return [Person(name) for name in names]
    
    
people = get_people("Alice", "Bob")

people[1].age

Overwriting people.py


In [34]:
mypy('people.py')

people.py:17: error: "Person" has no attribute "age"



# Dataclasses

In [35]:
from dataclasses import dataclass

@dataclass
class InventoryItem:
    """Class for keeping track of an item in inventory."""
    name: str
    unit_price: float
    quantity_on_hand: int = 0

    def total_cost(self) -> float:
        return self.unit_price * self.quantity_on_hand

### `__init__`

In [36]:
InventoryItem("Widget A", 1.99)

InventoryItem(name='Widget A', unit_price=1.99, quantity_on_hand=0)

In [37]:
InventoryItem("Widge B", 1.99, 300)

InventoryItem(name='Widge B', unit_price=1.99, quantity_on_hand=300)

In [38]:
InventoryItem("Widget C", 1.99, quantity_on_hand=400)

InventoryItem(name='Widget C', unit_price=1.99, quantity_on_hand=400)

In [39]:
InventoryItem(name="Widget D", unit_price=1.99, quantity_on_hand=500)

InventoryItem(name='Widget D', unit_price=1.99, quantity_on_hand=500)

In [40]:
try:
    InventoryItem(name="Widget E")
except TypeError as e:
    print(repr(e))

TypeError("__init__() missing 1 required positional argument: 'unit_price'")


### `__repr__`

In [41]:
item = InventoryItem("Widget", 1.99)

repr(item)

"InventoryItem(name='Widget', unit_price=1.99, quantity_on_hand=0)"

In [42]:
eval("InventoryItem(name='Widget', unit_price=1.99, quantity_on_hand=0)")

InventoryItem(name='Widget', unit_price=1.99, quantity_on_hand=0)

### `__eq__`

In [43]:
item_a = InventoryItem("Widget A", 1.99)
item_b = InventoryItem("Widget B", 1.99)
item_x = InventoryItem("Widget A", 1.99, quantity_on_hand=0)

In [44]:
item_a == item_b

False

In [45]:
item_a == item_x

True

### Ordered Dataclasses

In [46]:
@dataclass(order=True)
class Person:
    last_name: str
    first_name: str
        
    def __str__(self):
        return f'{self.first_name} {self.last_name}'

In [47]:
people = [
    Person(first_name="Alice", last_name="Smith"),
    Person(first_name="Bob", last_name="Smith"),
    Person(first_name="Carl", last_name="Davidson"),
]

for person in people:
    print(person)

Alice Smith
Bob Smith
Carl Davidson


In [48]:
people.sort()

In [49]:
for person in people:
    print(person)

Carl Davidson
Alice Smith
Bob Smith


### Frozen Dataclasses

In [50]:
@dataclass(frozen=True)
class Badge:
    number: int
        
badges = [Badge(1001), Badge(1002), Badge(1003)]

In [51]:
try:
    badges[1].number = 1004
except AttributeError as e:
    print(repr(e))

FrozenInstanceError("cannot assign to field 'number'")


### Field Customization 

In [52]:
from dataclasses import field

@dataclass(order=True)
class Person:
    name: str = field(compare=False)
    age: int
    
    def __str__(self):
        return f'{self.name} ({self.age})'

In [53]:
people = [
    Person("Alice Smith", 30),
    Person("Bob Smith", 25),
    Person("Carl Davidson", 41),
]

for person in people:
    print(person)

Alice Smith (30)
Bob Smith (25)
Carl Davidson (41)


In [54]:
people.sort()

In [55]:
for person in people:
    print(person)

Bob Smith (25)
Alice Smith (30)
Carl Davidson (41)


### ⚠️ Custom `__init__`

In [56]:
@dataclass
class Bill:
    subtotal: float
    tip: float = 0.0
    
    def __post_init__(self):
        self.total = self.subtotal + self.tip

In [57]:
bill = Bill(12.99, tip=3.00)

bill

Bill(subtotal=12.99, tip=3.0)

In [58]:
bill.total

15.99

### ⚠️ Mutable Default Values

In [59]:
from typing import List

try:
    
    @dataclass
    class Group:
        members: List[Person] = []
            
except ValueError as e:
    print(repr(e))

ValueError("mutable default <class 'list'> for field members is not allowed: use default_factory")


In [60]:
from dataclasses import field

@dataclass
class Group:
    members: List[Person] = field(default_factory=list)

In [61]:
group = Group()

group.members.append(people[0])

group

Group(members=[Person(name='Bob Smith', age=25)])

# Serialization (with datafiles)

In [62]:
# pip install datafiles==0.2b6

In [63]:
%%sh

rm -rf items

In [64]:
from dataclasses import dataclass

from datafiles import datafile

@datafile('items/{self.name}.yml')
@dataclass
class MyInventoryItem:
    """Class for keeping track of an item in inventory."""
    name: str
    unit_price: float
    quantity_on_hand: int = 0

    def total_cost(self) -> float:
        return self.unit_price * self.quantity_on_hand

In [65]:
item = MyInventoryItem("widget", 1.99)

INFO: datafiles.managers: Saving 'MyInventoryItem' object to 'items/widget.yml'


In [66]:
%%sh

cat items/widget.yml

unit_price: 1.99


In [67]:
item.quantity_on_hand += 100

INFO: datafiles.managers: Saving 'MyInventoryItem' object to 'items/widget.yml'


In [68]:
%%sh

cat items/widget.yml

unit_price: 1.99
quantity_on_hand: 100


In [69]:
%%writefile items/widget.yml

unit_price: 2.5  # was 3.0
quantity_on_hand: 100

Overwriting items/widget.yml


In [70]:
item.unit_price

INFO: datafiles.managers: Loading 'MyInventoryItem' object from 'items/widget.yml'


2.5

In [71]:
from datafiles import Missing

item = MyInventoryItem("widget", Missing)

assert item.unit_price == 2.5
assert item.quantity_on_hand == 100

item

INFO: datafiles.managers: Loading 'MyInventoryItem' object from 'items/widget.yml'


MyInventoryItem(name='widget', unit_price=2.5, quantity_on_hand=100)